In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(model_name="gpt-4o-mini")

In [26]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everthing you are asked You need to answer with a comma separated list of max {max_items} in list. Do NOT reply with anything else."),
    ("human", "{question}")
])


In [27]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?",
})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']